Para correr el programa, aprieta el botón para correr todas las celdas, por favor se paciente, funciona :)

In [1]:
print('Por favor espera unos minutos mientras se carga el archivo...')

Por favor espera unos minutos mientras se carga el archivo...


In [2]:
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import display_html
import warnings
warnings.filterwarnings("ignore")

raw_tweets = pd.read_json(r'farmers-protest-tweets-2021-03-5.json', lines=True)
raw_tweets = raw_tweets[raw_tweets['lang']=='en']

In [3]:
users = json_normalize(raw_tweets['user'])
users.drop(['description', 'linkTcourl'], axis=1, inplace=True)
users.rename(columns={'id':'userId', 'url':'profileUrl'}, inplace=True)

user_id = []
for user in raw_tweets['user']:
    uid = user['id']
    user_id.append(uid)
raw_tweets['userId'] = user_id

In [4]:
cols = ['id', 'retweetCount', 'userId', 'url', 'date', 'renderedContent']
tweets = raw_tweets[cols]
tweets.rename(columns={'id':'tweetId', 'url':'tweetUrl'}, inplace=True)
tweets = pd.DataFrame(tweets)
tweets.drop_duplicates(subset=['tweetId'], inplace=True)

In [5]:
def get_top_tweets():
  top_tweets = tweets.nlargest(10, 'retweetCount')
  print('\nTop 10 tweets:\n')
  return top_tweets

In [6]:
def get_top_users():
  print("\nCargando top users, esto tardará un momento :)")
  users_tweets = tweets[['tweetId', 'tweetUrl','renderedContent', 'userId']].merge(users[['userId','displayname']], on='userId')
  top_users = users_tweets.groupby(['displayname']).size().reset_index(name='tweets').sort_values(by='tweets', ascending=False)
  print('\nTop 10 users:\n')
  return top_users

In [7]:
def get_top_days():
  print("\nCargando top days, esto tardará un momento :)")
  new_tweets = tweets
  new_tweets['date'] = new_tweets['date'].dt.strftime('%m-%d-%Y')
  top_days = new_tweets.groupby(['date']).size().reset_index(name='tweets').sort_values(by='tweets', ascending=False)
  print('\nTop 10 days:\n')
  return top_days

In [8]:
def get_top_hashtags():
  print("\nCargando top hashtags, esto tardará un momento :)")
  top_hashtags = tweets.renderedContent.str.extractall(r'(\#\w+)')[0].value_counts()
  top_hashtags.sort_values(ascending=False)
  print('\nTop 10 hashtags:\n')
  return top_hashtags.head(10) 

In [9]:
def show_table(df):
  display_html(df.head(10))

In [10]:
def main():
  print('\nArchivo cargado.')
  exit_program = False
  while not exit_program:
    print(
      '''
      Elige el número de la función a ejecutar:
      
      [ 1 ] : Top 10 tweets
      [ 2 ] : Top 10 users
      [ 3 ] : Top 10 days
      [ 4 ] : Top 10 hashtags
      [ 5 ] : Salir del programa
      '''
    )
    input_correct = False
    while not input_correct:
      user_input = input('Ingresa un número: ')
      try:
        value = int(user_input)
        if value < 1 or value > 5:
          print('Error: Ingresa un número entre 1 y 5')
        else:
          input_correct = True
          if value == 1:
            show_table(get_top_tweets())
          elif value == 2:
            show_table(get_top_users())
          elif value == 3:
            show_table(get_top_days())
          elif value == 4:
            print(get_top_hashtags())
          elif value == 5:
            print('Programa finalizado.')
            exit_program = True
      except ValueError:
        print('Error: Recuerda ingresar un número')
        continue

In [11]:
main()


Archivo cargado.

      Elige el número de la función a ejecutar:
      
      [ 1 ] : Top 10 tweets
      [ 2 ] : Top 10 users
      [ 3 ] : Top 10 days
      [ 4 ] : Top 10 hashtags
      [ 5 ] : Salir del programa
      
Ingresa un número: 1

Top 10 tweets:



,tweetId,retweetCount,userId,tweetUrl,date,renderedContent
408128,1356625889602199552,315547,79293791,https://twitter.com/rihanna/status/13566258896...,2021-02-02 15:29:51+00:00,why aren’t we talking about this?! #FarmersPro...
395142,1356694884615340037,103957,1006419421244678144,https://twitter.com/GretaThunberg/status/13566...,2021-02-02 20:04:01+00:00,We stand in solidarity with the #FarmersProtes...
266196,1357282507616645122,67694,1006419421244678144,https://twitter.com/GretaThunberg/status/13572...,2021-02-04 10:59:01+00:00,I still #StandWithFarmers and support their pe...
366579,1356848397899112448,35921,2835653131,https://twitter.com/miakhalifa/status/13568483...,2021-02-03 06:14:01+00:00,"“Paid actors,” huh? Quite the casting director..."
372793,1356827705161879553,26972,2835653131,https://twitter.com/miakhalifa/status/13568277...,2021-02-03 04:51:48+00:00,What in the human rights violations is going o...
314192,1357048037302960129,23251,733170759829327874,https://twitter.com/TeamJuJu/status/1357048037...,2021-02-03 19:27:19+00:00,"Happy to share that I’ve donated $10,000 to pr..."
215034,1357755699162398720,20132,805185025,https://twitter.com/BobBlackman/status/1357755...,2021-02-05 18:19:19+00:00,There has been much social media coverage arou...
398011,1356681136655769605,18744,1134059457191776257,https://twitter.com/vanessa_vash/status/135668...,2021-02-02 19:09:23+00:00,Farmers feed the world. Fight for them. Protec...
325261,1357009721090138112,17368,272616327,https://twitter.com/kylekuzma/status/135700972...,2021-02-03 16:55:04+00:00,Should be talking about this! #FarmersProtest\...
163689,1359013362881994752,15677,104856942,https://twitter.com/AmandaCerny/status/1359013...,2021-02-09 05:36:49+00:00,To all of my influencer/celeb friends- read up...



      Elige el número de la función a ejecutar:
      
      [ 1 ] : Top 10 tweets
      [ 2 ] : Top 10 users
      [ 3 ] : Top 10 days
      [ 4 ] : Top 10 hashtags
      [ 5 ] : Salir del programa
      
Ingresa un número: 2

Cargando top users, esto tardará un momento :)

Top 10 users:



,displayname,tweets
21541,Harjot Singh,50905719
76395,ਕਿਸਾਨAndolajivi ravinder kaur चोकीदार ही कातिल है,4359744
26144,"Jaspal Kaur Bains.For love of Punjab,Sikhi & I...",3964081
27464,Jot,3389564
59968,Theinactiveactivist,3250809
79541,🍊raman🚜,2965284
26757,Jaz 🇨🇦🌾ਗਰਮ ਖਿਆਲੀ 📌,2223081
26308,JassG,2125764
29904,Kisan Bot🚜🌾,2111209
69678,mohd khaliquzzama,2090916



      Elige el número de la función a ejecutar:
      
      [ 1 ] : Top 10 tweets
      [ 2 ] : Top 10 users
      [ 3 ] : Top 10 days
      [ 4 ] : Top 10 hashtags
      [ 5 ] : Salir del programa
      
Ingresa un número: 3

Cargando top days, esto tardará un momento :)

Top 10 days:



,date,tweets
2,02-03-2021,83403
3,02-04-2021,58300
4,02-05-2021,33165
1,02-02-2021,28440
5,02-06-2021,22298
6,02-07-2021,11244
8,02-09-2021,9269
7,02-08-2021,8863
9,02-10-2021,7938
10,02-11-2021,5668



      Elige el número de la función a ejecutar:
      
      [ 1 ] : Top 10 tweets
      [ 2 ] : Top 10 users
      [ 3 ] : Top 10 days
      [ 4 ] : Top 10 hashtags
      [ 5 ] : Salir del programa
      
Ingresa un número: 4

Cargando top hashtags, esto tardará un momento :)

Top 10 hashtags:

#FarmersProtest              404687
#IStandWithFarmers            15713
#farmersprotest               15378
#IndianFarmersHumanRights     11934
#FarmersAreIndia              10985
#StandWithFarmers             10612
#Rihanna                       9088
#FarmersProtests               8707
#Farmers                       6541
#shameonbollywood              6222
Name: 0, dtype: int64

      Elige el número de la función a ejecutar:
      
      [ 1 ] : Top 10 tweets
      [ 2 ] : Top 10 users
      [ 3 ] : Top 10 days
      [ 4 ] : Top 10 hashtags
      [ 5 ] : Salir del programa
      
Ingresa un número: 5
Programa finalizado.
